In [1]:
!pip install mlflow

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/29.0 MB ? eta -:--:--
    --------------------------------------- 0.5/29.0 MB 5.6 MB/s eta 0:00:06
   -- ------------------------------------- 1.8/29.0 MB 5.0 MB/s eta 0:00:06
   --- ------------------------------------ 2.9/29.0 MB 4.9 MB/s eta 0:00:06
   ----- ---------------------------------- 3.9/29.0 MB 4.9 MB/s eta 0:00:06
   ------- -------------------------------- 5.2/29.0 MB 5.1 MB/s eta 0:00:05
   -------- ------------------------------- 6.3/29.0 MB 5.1 MB/s eta 0:00:05
   ---------- ----------------------------- 7.3/29.0 MB 5.2 MB/s eta 0:00:05
   ----------- ---------------------------- 8.7/29.0 MB 5.1 MB/s eta 0:00:04
   ------------- -------------------------- 9.7/29.0 MB 5.1 MB/s eta 0:00:04
   -------------- ------------------------- 10.5/29.0 MB 5.0 MB/s eta 0:00:04
   ---------------- ----------------------- 11.8/29.0 MB 5.0 MB/s eta 0:00:04
   

DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# 📌 Step 1: Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
import mlflow
import mlflow.sklearn
import joblib
import time

# 📌 Step 2: Set MLflow Tracking URI
mlflow.set_tracking_uri(r"file:///E://UNIVERSITY//Semester 6//Big Data Analytics//LAB//ML_Assignment//ML//mlruns")

# 📌 Step 3: Load the crop dataset
df = pd.read_csv("Crop_recommendation.csv")

# Remove unnamed columns if they exist
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# 📌 Step 4: Encode label (crop name)
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

# 📌 Step 5: Feature & target split
X = df.drop('label', axis=1)
y = df['label']

# 📌 Step 6: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 Step 7: Start MLflow experiment
mlflow.set_experiment("Crop_Recommendation_Experiment")

with mlflow.start_run():
    start_time = time.time()

    # Model training
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Prediction
    y_pred = model.predict(X_test)

    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    duration = time.time() - start_time

    # ✅ Log to MLflow
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("training_time", duration)

    # ✅ Save model and label encoder
    mlflow.sklearn.log_model(model, "model")
    joblib.dump(model, "model.pkl")
    joblib.dump(le, "label_encoder.pkl")

    print("✅ Model trained and saved successfully.")


2025/06/02 01:37:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Model trained and saved successfully.


In [6]:

import pandas as pd
import numpy as np
df = pd.read_csv("Crop_recommendation.csv")  # <-- Make sure crop.csv is in the same folder
df.describe()

,Nitrogen,phosphorus,potassium,temperature,humidity,ph,rainfall,Unnamed: 8,Unnamed: 9
count,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,0.0,0.0
mean,50.551818,53.362727,48.149091,25.616244,71.481779,6.469480,103.463655,NaN,NaN
std,36.917334,32.985883,50.647931,5.063749,22.263812,0.773938,54.958389,NaN,NaN
min,0.000000,5.000000,5.000000,8.825675,14.258040,3.504752,20.211267,NaN,NaN
25%,21.000000,28.000000,20.000000,22.769375,60.261953,5.971693,64.551686,NaN,NaN
50%,37.000000,51.000000,32.000000,25.598693,80.473146,6.425045,94.867624,NaN,NaN
75%,84.250000,68.000000,49.000000,28.561654,89.948771,6.923643,124.267508,NaN,NaN
max,140.000000,145.000000,205.000000,43.675493,99.981876,9.935091,298.560117,NaN,NaN


In [7]:
df.head()

,Nitrogen,phosphorus,potassium,temperature,humidity,ph,rainfall,label,Unnamed: 8,Unnamed: 9
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,NaN,NaN
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice,NaN,NaN
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice,NaN,NaN
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice,NaN,NaN
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice,NaN,NaN
